# Upload Data to Hub Contracts

In [1]:
import json
import base64
import requests

from cyber_sdk.client.lcd import LCDClient
from cyber_sdk.key.mnemonic import MnemonicKey
from cyberutils.contract import execute_contract

from ..config import LCD_CLIENT, NODE_LCD_URL, WALLET_SEED, WALLET_ADDRESS

mk = MnemonicKey(mnemonic=WALLET_SEED)
WALLET = LCD_CLIENT.wallet(mk)

MEMO = 'update entry'

HUB_CONTRACTS = {
    'protocols': 'bostrom12yqsxh82qy3dz6alnmjhupyk85skgeqznzxv92q99hqtyu7vvdsqgwjgvz',
    'networks': 'bostrom1lpn69a74ftv04upfej8f9ay56pe2zyk48vzlk49kp3grysc7u56qq363nr',
    'channels': 'bostrom15tx5z779rdks07sg774ufn8q0a9x993c9uwmr6ycec78z6lfrmkqyjnfge',
    'tokens': 'bostrom15phze6xnvfnpuvvgs2tw58xnnuf872wlz72sv0j2yauh6zwm7cmqqpmc42',
    'skills': 'bostrom1ejm29m279e8x4ltf3ksqm443fsdql9rmtuyth6vssmxl72af5yzqrt2mku',
    'contracts': 'bostrom1cfa46qr4ca4n75eys534fzmdtgce9zscvlzfq8fgt4sk6gc250hssks43s'}

UPLOAD_HUB = {
    'protocols': False,
    'networks': False,
    'channels': True,
    'tokens': False,
    'skills': False,
    'contracts': False
}

REQUIRED_KEYS = {
    'protocols': ['id', 'data_type'],
    'networks': ['id'],
    'channels': ['id', 'active'],
    'tokens': ['id'],
    'skills': ['id', 'new_owner'],
    'contracts': ['id']
}

INDEX_KEYS = {
    'protocols': ['data_type'],
    'networks': ['chain_id', 'genesis_hash', 'protocol'],
    'channels': ['source_chain_id', 'destination_chain_id', 'source_channel_id', 'destination_channel_id'],
    'tokens': ['chain_id', 'contract'],
    'skills': ['network', 'neuron', 'protocol'],
    'contracts': ['address', 'chain_id']
}

UPLOAD_DATA_FILES = {k: f'data/{k}.csv' for k in HUB_CONTRACTS.keys()}
TX_FILES = {k: f'txs/tx_update_hub_{k}.json' for k in HUB_CONTRACTS.keys()}

FEE_DENOM = 'boot'

## Update Entries

In [2]:
def contract_query(
        contract_address: str,
        query: dict,
        node_lcd_url: str = NODE_LCD_URL) -> dict:

    _query_msg = base64.b64encode(json.dumps(query).encode("utf-8")).decode("utf-8")
    _query = f'{node_lcd_url}/cosmwasm/wasm/v1/contract/{contract_address}/smart/{_query_msg}'
    return requests.get(_query).json()


def get_data_for_update(
        contract_name: str,
        contracts: dict = HUB_CONTRACTS,
        index_keys: dict = INDEX_KEYS) -> list[dict]:

    _contract_entries = contract_query(
        contract_address = contracts[contract_name],
        query={'get_entries': {}})['data']['entries']
    with open(f'data/{contract_name}.json', 'r') as _file:
        _data_for_update = json.load(_file)

    _data_for_update_selected = []
    for _contract_entry in _contract_entries:
        _data_for_update_item_list = \
            [data_for_update_item for data_for_update_item in _data_for_update
                if dict((k, data_for_update_item.get(k)) for k in index_keys[contract_name]) == dict((k, _contract_entry.get(k)) for k in index_keys[contract_name])]
        if _data_for_update_item_list:
            assert len(_data_for_update_item_list) == 1
            _data_for_update_item = _data_for_update_item_list[0]
            if {k: v for k, v in _data_for_update_item.items() if k not in _contract_entry.keys() or _contract_entry[k] != v}:
                _data_for_update_selected.append(dict({"id": _contract_entry['id']}, **_data_for_update_item))
    return _data_for_update_selected


In [3]:
def hub_update(
        tx_file_name: str,
        items_for_update: list[dict],
        contract_address: str,
        required_keys: list[str],
        sender: str = WALLET_ADDRESS,
        lcd_client: LCDClient = LCD_CLIENT,
        fee_denom: str = FEE_DENOM,
        gas: int = 5_000_000,
        memo: str = MEMO) -> dict:
    _execute_msgs = []
    for _item_for_update in items_for_update:
        assert all(_item in _item_for_update.keys() for _item in required_keys)
        _execute_msg = {'UpdateEntry': _item_for_update}
        _execute_msgs.append(_execute_msg)
    print(_execute_msgs)
    print(contract_address)
    print(sender)
    _tx = execute_contract(
        wallet=WALLET,
        execute_msgs=_execute_msgs,
        contract_address=contract_address,
        sender=sender,
        lcd_client=lcd_client,
        fee_denom=fee_denom,
        gas=gas,
        sign_and_broadcast_tx=True,
        memo=memo
    )
    with open(tx_file_name, 'w') as _tx_file:
        _tx_file.write(json.dumps(_tx.to_data()))
    return _tx.to_data()

In [4]:
tx_data = {}
for contract_name in HUB_CONTRACTS.keys():
    if UPLOAD_HUB.get(contract_name, False):
        data_for_update = get_data_for_update(contract_name=contract_name)
        if data_for_update:
            tx_data[contract_name] = hub_update(
                tx_file_name=TX_FILES[contract_name],
                items_for_update=data_for_update,
                contract_address=HUB_CONTRACTS[contract_name],
                required_keys=REQUIRED_KEYS[contract_name],
                gas=20_000_000
            )

[{'UpdateEntry': {'id': 5, 'active': 'true', 'destination_chain_id': 'gravity-bridge-3', 'destination_channel_id': 'channel-103', 'source_chain_id': 'bostrom', 'source_channel_id': 'channel-12', 'explorer_url': 'https://mintscan.io/gravity-bridge/txs/{txHash}'}}, {'UpdateEntry': {'id': 16, 'active': 'false', 'destination_chain_id': 'gravity-bridge-3', 'destination_channel_id': 'channel-78', 'source_chain_id': 'bostrom', 'source_channel_id': 'channel-7', 'explorer_url': 'https://mintscan.io/gravity-bridge/txs/{txHash}'}}]
bostrom15tx5z779rdks07sg774ufn8q0a9x993c9uwmr6ycec78z6lfrmkqyjnfge
bostrom1cj8j6pc3nda8v708j3s4a6gq2jrnue7j857m9t


In [5]:
tx_data['channels']

{'height': 10361422,
 'txhash': '1F64912FDA7C24E9017EFA4B8CBCA3E27069F6384616D7C0F85CC7F4D2D613C5',
 'raw_log': '[{"events":[{"type":"execute","attributes":[{"key":"_contract_address","value":"bostrom15tx5z779rdks07sg774ufn8q0a9x993c9uwmr6ycec78z6lfrmkqyjnfge"}]},{"type":"message","attributes":[{"key":"action","value":"/cosmwasm.wasm.v1.MsgExecuteContract"},{"key":"module","value":"wasm"},{"key":"sender","value":"bostrom1cj8j6pc3nda8v708j3s4a6gq2jrnue7j857m9t"}]},{"type":"wasm","attributes":[{"key":"_contract_address","value":"bostrom15tx5z779rdks07sg774ufn8q0a9x993c9uwmr6ycec78z6lfrmkqyjnfge"},{"key":"method","value":"execute_update_entry"},{"key":"updated_entry_id","value":"5"}]}]},{"msg_index":1,"events":[{"type":"execute","attributes":[{"key":"_contract_address","value":"bostrom15tx5z779rdks07sg774ufn8q0a9x993c9uwmr6ycec78z6lfrmkqyjnfge"}]},{"type":"message","attributes":[{"key":"action","value":"/cosmwasm.wasm.v1.MsgExecuteContract"},{"key":"module","value":"wasm"},{"key":"sender"